In [ ]:
import os
import pandas as pd
# Define the base path and the range of years
base_path = '../team/'
start_year = 1997
end_year = 2025
frames = []
# Loop through each year and list files in the corresponding directory
for year in range(start_year, end_year + 1):
    # Construct the path for the current year
    year_path = os.path.join(base_path, str(year))
    
    # Check if the directory exists
    if os.path.isdir(year_path):
        print(f"\nFiles in directory {year_path}:")
        
        # List all files in the directory
        for filename in os.listdir(year_path):
            file_path = os.path.join(year_path, filename)
            # Check if it is a file before printing
            if 'vs' not in filename and 'avg' not in filename:
                if '.csv' in filename:
                    
                    
                    df=pd.read_csv(file_path)
                    if len(df)>0:
                        print(filename)
                        df['year']=year
                        frames.append(df)

    else:
        print(f"\nDirectory {year_path} does not exist.")

master=pd.concat(frames)
master


Files in directory ../team/1997:
1610612756.csv
1610612765.csv
1610612757.csv
1610612749.csv
1610612738.csv
1610612747.csv
1610612753.csv
1610612742.csv
1610612743.csv
1610612754.csv
1610612766.csv
1610612759.csv
1610612746.csv
1610612763.csv
1610612764.csv
1610612748.csv
1610612737.csv
1610612760.csv
1610612762.csv
1610612752.csv
1610612741.csv
1610612750.csv
1610612751.csv
1610612739.csv
1610612745.csv
1610612744.csv
1610612758.csv
1610612761.csv
1610612755.csv

Files in directory ../team/1998:
1610612756.csv
1610612765.csv
1610612757.csv
1610612749.csv
1610612738.csv
1610612747.csv
1610612753.csv
1610612742.csv
1610612743.csv
1610612754.csv
1610612766.csv
1610612759.csv
1610612746.csv
1610612763.csv
1610612764.csv
1610612748.csv
1610612737.csv
1610612760.csv
1610612762.csv
1610612752.csv
1610612741.csv
1610612750.csv
1610612751.csv
1610612739.csv
1610612745.csv
1610612744.csv
1610612758.csv
1610612761.csv
1610612755.csv

Files in directory ../team/1999:
1610612756.csv
1610612765.cs

In [ ]:
master['SHOT_ATTEMPTED_FLAG'].value_counts()
master['SHOT_MADE_FLAG'].value_counts()

In [ ]:
shots = master.groupby(['year','SHOT_ZONE_RANGE','SHOT_DISTANCE']).sum(numeric_only=True)[['SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG']].reset_index()

shots.rename(columns={'SHOT_ATTEMPTED_FLAG':'FGA','SHOT_MADE_FLAG':'FGM'},inplace=True)
shots

In [ ]:
for year in range(1997,2026):
    shot_distance=shots[shots.year==year].reset_index()
    shot_distance.drop(columns='year',inplace=True)
    shot_distance.to_csv(str(year)+'.csv',index=False)

In [ ]:
master['SHOT_ZONE_RANGE'].unique()